# AutoCAD MegaIntellegence Project
  Предназначено для ускорения работы с конструкторскими чертежами. Машинное обучение позволит убрать рутинные операции. Программа должна проставлять размеры на точки крепления к приборам, с учётом всех требований. Это первая задача. С имеющейся базой можно расширить применение на другие задачи работы с dwg чертежами.
  
  Проект читает $хорошие$ чертежи, обучается на существующих позициях размеров. X - линии в слое с точками крепления, пересечение с линиями прибора. y - базовые точки размеров, положение текста
  
  Первая версия: будет один вид, нужно проставить все размеры на 1 виде.
  
$$успех = усердие + время$$

In [1]:

import win32com.client
import comtypes.client
import numpy as np
import pandas as pd
import math
import array

app = comtypes.client.GetActiveObject('AutoCAD.Application')
#app = win32com.client.Dispatch("AutoCAD.Application")
activedoc = app.ActiveDocument
msp =activedoc.ModelSpace

In [2]:
for blk in activedoc.Blocks:
    print(blk)
#    for ent in blk:
#         print(ent)

<POINTER(IAcadModelSpace) ptr=0x29e06a33468 at 29e0853f848>
<POINTER(IAcadPaperSpace) ptr=0x29e04b99d38 at 29e0848c948>
<POINTER(IAcadBlock) ptr=0x29e04b3bb08 at 29e0848cb48>
<POINTER(IAcadBlock) ptr=0x29e04b990f8 at 29e0848cbc8>
<POINTER(IAcadBlock) ptr=0x29e06a332a8 at 29e0848cac8>
<POINTER(IAcadBlock) ptr=0x29e04b3b638 at 29e0853f848>
<POINTER(IAcadBlock) ptr=0x29e04b99b78 at 29e0848c6c8>
<POINTER(IAcadBlock) ptr=0x29e06a33d98 at 29e0853f848>


In [30]:
# set variables
df = pd.DataFrame(columns ={'device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y'})
df = df[['device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y']]
dim_aligned, dim_rotated, blockref, i,= 0,0,0,0
# view each object in ModelSpace
for item in msp:
    #TODO: need to check if that actually device block
    #if it's a device:
    if item.ObjectName == 'AcDbBlockReference':
        #find BLOCK object by given BlockRef
        block = activedoc.Blocks.Item(item.Name)
        print (f'Block name: {block}')
        for element in block:
            if element.Layer =="attachment point":
                # fill the dataframe with the coordinates of the attachment points
                  df = df.append({'device': i, \
                                'start_att_line_X': element.StartPoint[0] + item.InsertionPoint[0], \
                                'start_att_line_Y': element.StartPoint[1] + item.InsertionPoint[1], \
                                'end_att_line_X': element.EndPoint[0] + item.InsertionPoint[0],\
                                'end_att_line_Y': element.EndPoint[1] + item.InsertionPoint[1]},  ignore_index=True)
        i+=1
                
df['device'] = df['device'].astype(int)


Block name: <POINTER(IAcadBlock) ptr=0x29e07119098 at 29e09a0e7c8>
Block name: <POINTER(IAcadBlock) ptr=0x29e07118a78 at 29e09a0e448>
Block name: <POINTER(IAcadBlock) ptr=0x29e06a33d98 at 29e09a0e0c8>


In [31]:
df

,device,start_att_line_X,end_att_line_X,start_att_line_Y,end_att_line_Y
0,0,468.643810,473.643810,569.910725,569.910725
1,0,471.143810,471.143810,572.410725,567.410725
2,0,538.643810,543.643810,569.910725,569.910725
3,0,541.143810,541.143810,572.410725,567.410725
4,0,468.643810,473.643810,539.910725,539.910725
5,0,471.143810,471.143810,542.410725,537.410725
6,0,538.643810,543.643810,539.910725,539.910725
7,0,541.143810,541.143810,542.410725,537.410725
8,1,656.799864,661.799864,539.910725,539.910725
9,1,656.799864,661.799864,569.910725,569.910725


In [32]:
# Length of attraction line must be 5mm
# Is the line in this row horisontal?
df['horizontal']= (df.iloc[:,1] == df.iloc[:,2])
# Find center of this line at axis X. If line is horizontal...
df['point_X'] =np.where(df['horizontal'], \
                        # any coordinate is suitable
                        df.iloc[:, 1], \
                        # otherwise, take the middle of the line by X
                        (np.minimum(df.iloc[:, 1], df.iloc[:, 2]) +2.5))

#same for Y
df['point_Y'] =np.where(~ df['horizontal'], \
                        df.iloc[:, 3], \
                        (np.minimum(df.iloc[:, 3], df.iloc[:, 4]) +2.5))    
#df

In [33]:
df2 = df.drop_duplicates(['point_X', 'point_Y'])
df2 = df2.drop(columns = ['start_att_line_X', 'end_att_line_X', 'start_att_line_Y', 'end_att_line_Y', 'horizontal'])
df2

,device,point_X,point_Y
0,0,471.143810,569.910725
2,0,541.143810,569.910725
4,0,471.143810,539.910725
6,0,541.143810,539.910725
8,1,659.299864,539.910725
9,1,659.299864,569.910725
10,2,475.100512,460.108155
11,2,545.100512,460.108155


In [ ]:
#df
    #print(f'got an item: {item}')
    #print(f'Object name: {item.ObjectName}')
    #print(item._methods_)
    
#     if item.ObjectName =='AcDbRotatedDimension':
#         #print('found aligned dimention.')
#         #print(f'Object layer name: {item.Layer}\n')       
#         dim_rotated = item
#     elif item.ObjectName == 'AcDbAlignedDimension':
#         #print('found rotated dimention\n')
#         dim_aligned = item
        
    #explode blocks
#    elif item.ObjectName == 'AcDbBlockReference':       
#         try:
#             print(f'Found a block.\n')
#             block = item
#             block.Explode()
#         except:
#             pass    # here we have some VARIANT bug. So just proceed.
#         i+=1

In [7]:
def FindAttachmentPoint(data):
    """This function converts given data to the coordinates of attachment points. 
    
    data: dataframe with coordinates of lines in layer 'attachment point'
    
    return
    """
    #FindAttachmentPoint(df[df['device']==device])
    print(f'data:\n{data}')
    print(data.iloc[[0]])


def MakeDimVertical(msp, point1, point2, loc):
    """This function make vertical dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass
    

def MakeDimHorizontal(msp, point1, point2, loc):
    """This function make horizontal dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass

def ProcessAttachmentPoints(msp, objects):
    """This function make coorginates of attachment points.
    
    msp:  model space
    objects:  lines from specified layer
    """
    pass


In [8]:
pt = array.array('d', [0,0,0]) # to convert in variant
point = mSp.AddPoint(pt)
# Add a LINE
pt1 = array.array('d', [0.0,0.0,0]) # start point
pt2 = array.array('d', [20.0,20.0,0]) # end point
line = mSp.AddLine(pt1, pt2)

NameError: name 'mSp' is not defined

In [ ]:
s1 =  array.array('d', [500.0, 800.0, 0.0])
s2 =  array.array('d', [500.0, 1200.0, 0.0])
loc = array.array('d', [500.0, 1000.0, 0.0])
new_dim = mSp.AddDimAligned(s1, s2, loc)
new_dim.update()

In [ ]:
dim.


In [ ]:
df = pd.read_excel('E://Python/RubinProject/Journal.xlsx')

In [ ]:
make_dim_vertical()

In [ ]:
df = df.rename({'Имя прибора': 'prib_name',
               'Индекс по схеме': 'shema_index'},
               axis=1)

In [ ]:
# import win32com.client
# acad = win32com.client.Dispatch("AutoCAD.Application")

# doc = acad.ActiveDocument   # Document object


# # iterate trough all objects (entities) in the currently opened drawing
# # and if its a BlockReference, display its attributes and some other things.
# for entity in acad.ActiveDocument.ModelSpace:
#     name = entity.EntityName
#     if name == 'AcDbBlockReference':
#         HasAttributes = entity.HasAttributes
#         if HasAttributes:
#             print(entity.Name)
#             print(entity.Layer)
#             print(entity.ObjectID)
#             for attrib in entity.GetAttributes():
#                 print("  {}: {}".format(attrib.TagString, attrib.TextString))
                
#                 # update text
#                 attrib.TextString = 'modified with python'
#                 attrib.Update()

Фичи размеров в качестве таргетов:
- TextPosition (rotated размеры тоже их имеют)

Запасной вариант- взрывать кхерам все размеры, вытаскивать параметры линий, и таким образом получать координаты точек привязки размера, а заодно и координаты текста. Наверное, придётся так и сделать, чтобы обеспечить непересекаемоть размеров и других линий чертежа, типа линий габаритки

todo:

точка крепления может быть не посередине линии соответствующего слоя, а где-то на пересечении с линией прибора. Сделать проверку на небольшое отклонение и выдать сообщение чтобы поправили чертёж